- 1) One hot encoding modelin görmediği veriye nasıl uygulanır?
- 2) Labelencoding'de ilgili kolon için ölçeklendirme nasıl yapılır? (Verinin doğru etkisiyle sayısal olarak dönüştürülmesi)
- 3) Imbalance datasette train test split yaparken neleri göz önünde bulundurmalıyız?
- 4) Validation dataseti (modelin görmediği) nasıl oluşturulur ve nasıl predict etmeye hazır hale getirilir?
- 5) predict_proba metoduyla oran nasıl hesaplanır ve threshold nasıl değiştirilir?
- 6) Fraud case'i üzerinde train&test&validation split, encoding, scaling,modelleme çalışmaları Python'da yapılarak, modelin görmediği dataset üzerinde başarılı sonuç alacak bir model örneği yapılmalı.

## soru 1

**Soru:** One hot encoding modelin görmediği veriye nasıl uygulanır?

One hot encodingi modelin görmediği veriye yani validationa uygularken dikkat etmemiz gereken şey model ve validation setindeki aynı isimli kolonların farklı sayıda unique değere sahip olabileceğidir. Buna dikkat etmezsek modelimizin ve validationımızın farklı sayıda kolonları oluşacağından modelimiz çalışmaz. Bu durumun nasıl düzeltileceğini derste yapıp hata aldığımız örnek üzerinden gösterdim.

Aşağıdaki ilk 13 kod bloğunda derste yaptığımız örneği buraya aktardım (o yüzden bu kısımlara yorum satırı koymadım), sonrası hatayı düzelttiğim kısım. Derstekinden farklı olarak unique değiken sayısı 1000 olan (yani her satırda farklı olan) bir kategorik kolonu (incident_location) attığım için encode sonrası oluşan **kolon sayısı** ve **model - validation setleri arasındaki unique değişken farkı oldukça azaldı**.

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import plotly.express as px
import plotly.graph_objects as go


#for quick viz
import seaborn as sns

#ml
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
import random

In [2]:
df = pd.read_csv('auto_insurance_csv.csv')
df = df.drop(['_c39','Unnamed: 0'], axis = 1)
df['umbrella_limit'] = df.umbrella_limit.fillna("9999")
df['police_report_available'] = df.police_report_available.fillna("MISSING")
df['policy_csl'] = df.policy_csl.fillna("MISSING")
df['policy_bind_date'] = pd.to_datetime(df['policy_bind_date'])
df['incident_date'] = pd.to_datetime(df['incident_date'])
df['claim_day_of_policy'] = (df.incident_date -  df.policy_bind_date).dt.days
df['location_check'] = np.nan
df['location_check'] = np.where(df['policy_state'] == df['incident_state'], True, False)
df['fraud_reported'] = df['fraud_reported'].str.replace('Y', '1')
df['fraud_reported'] = df['fraud_reported'].str.replace('N', '0')
df['fraud_reported'] = df['fraud_reported'].astype(int)
df['umbrella_limit'] = df.umbrella_limit.astype(str)
umbrealla = df['umbrella_limit'].unique()
for umb in umbrealla:
  if (umb != '0.0') & (umb != '9999'):
    df['umbrella_limit'] = df['umbrella_limit'].str.replace(umb, 'other')
    
hobbies = df['insured_hobbies'].unique()
for hobby in hobbies:
  if (hobby != 'chess') & (hobby != 'cross-fit'):
    df['insured_hobbies'] = df['insured_hobbies'].str.replace(hobby, 'other')

df['age'] = df.age.fillna(9999)
bin_labels = ['15-20', '21-25', '26-30', '31-35', '36-40', '41-45', '46-50', '51-55', '56-60', '61-65','9999']
bins = [15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 9999]

df['age_group'] = pd.cut(df['age'], bins = bins, labels = bin_labels, include_lowest = True)
bins = [0, 50, 100, 150, 200, 250, 300, 350, 400, 450, 500]
bin_labels = ['0-50','51-100','101-150','151-200','201-250','251-300','301-350','351-400','401-450','451-500']

df['months_as_customer_groups'] = pd.cut(df['months_as_customer'], bins = 10, labels = bin_labels, include_lowest= True)
bins = list(np.linspace(0,2500, 6, dtype = int))
bin_labels = ['very low', 'low', 'medium', 'high', 'very high']

df['policy_annual_premium_groups'] = pd.cut(df['policy_annual_premium'], bins = bins, labels=bin_labels)
bins = list(np.linspace(0,2000, 5, dtype = int))
bin_labels = ['0-500', '501-1000', '1001-1500', '1501-2000']

df['policy_deductable_group'] = pd.cut(df['policy_deductable'], bins = bins, labels = bin_labels)

df = df.drop(['age', 'months_as_customer', 'policy_deductable', 'policy_annual_premium'], axis = 1)
required_columns = ['incident_date','policy_state', 'policy_csl', 'umbrella_limit',
       'insured_zip', 'insured_sex', 'insured_education_level',
       'insured_occupation', 'insured_hobbies', 'insured_relationship',
       'capital-gains', 'capital-loss', 'incident_type', 'collision_type',
       'incident_severity', 'authorities_contacted', 'incident_state',
       'incident_city', 'incident_location', 'incident_hour_of_the_day',
       'number_of_vehicles_involved', 'property_damage', 'bodily_injuries',
       'witnesses', 'police_report_available', 'total_claim_amount','auto_make',
       'auto_model', 'auto_year', 'claim_day_of_policy',
       'location_check', 'age_group', 'months_as_customer_groups',
       'policy_annual_premium_groups', 'policy_deductable_group', 'fraud_reported']
df = df[required_columns]

<ipython-input-2-224acfb64d08>:18: FutureWarning: The default value of regex will change from True to False in a future version.
  df['umbrella_limit'] = df['umbrella_limit'].str.replace(umb, 'other')


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 36 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   incident_date                 1000 non-null   datetime64[ns]
 1   policy_state                  1000 non-null   object        
 2   policy_csl                    1000 non-null   object        
 3   umbrella_limit                1000 non-null   object        
 4   insured_zip                   1000 non-null   int64         
 5   insured_sex                   1000 non-null   object        
 6   insured_education_level       1000 non-null   object        
 7   insured_occupation            1000 non-null   object        
 8   insured_hobbies               1000 non-null   object        
 9   insured_relationship          1000 non-null   object        
 10  capital-gains                 1000 non-null   int64         
 11  capital-loss                  1

In [4]:
df["incident_location"].nunique()

1000

In [5]:
df["insured_zip"].nunique()

995

In [6]:
df = df.drop(["incident_location", "insured_zip"], axis = 1)

In [7]:
pd.set_option("display.max_columns", None)
df.head()

,incident_date,policy_state,policy_csl,umbrella_limit,insured_sex,insured_education_level,insured_occupation,insured_hobbies,insured_relationship,capital-gains,capital-loss,incident_type,collision_type,incident_severity,authorities_contacted,incident_state,incident_city,incident_hour_of_the_day,number_of_vehicles_involved,property_damage,bodily_injuries,witnesses,police_report_available,total_claim_amount,auto_make,auto_model,auto_year,claim_day_of_policy,location_check,age_group,months_as_customer_groups,policy_annual_premium_groups,policy_deductable_group,fraud_reported
0,2015-01-25,OH,250/500,0.0,MALE,MD,craft-repair,other,husband,53300,0,Single Vehicle Collision,Side Collision,Major Damage,Police,SC,Columbus,5,1,YES,1,2,YES,71610,Saab,92x,2004,100,False,46-50,301-350,medium,501-1000,1
1,2015-01-21,IN,250/500,other,MALE,MD,machine-op-inspct,other,other-relative,0,0,Vehicle Theft,?,Minor Damage,Police,VA,Riverwood,8,1,?,0,0,?,5070,Mercedes,E400,2007,3130,False,41-45,201-250,medium,1501-2000,1
2,2015-02-22,OH,MISSING,other,FEMALE,PhD,sales,other,own-child,35100,0,Multi-vehicle Collision,Rear Collision,Minor Damage,Police,NY,Columbus,7,3,NO,2,3,MISSING,34650,Dodge,RAM,2007,5282,False,26-30,101-150,medium,1501-2000,0
3,2015-01-10,IL,MISSING,other,FEMALE,PhD,armed-forces,other,unmarried,48900,-62400,Single Vehicle Collision,Front Collision,Major Damage,Police,OH,Arlington,5,1,?,1,2,NO,63400,Chevrolet,Tahoe,2014,8996,False,41-45,251-300,medium,1501-2000,1
4,2015-02-17,IL,MISSING,other,MALE,Associate,sales,other,unmarried,66000,-46000,Vehicle Theft,?,Minor Damage,None,NY,Arlington,20,1,NO,0,1,MISSING,6500,Accura,RSX,2009,256,False,41-45,201-250,high,501-1000,0


In [8]:
pd.reset_option("display.max_columns")
df.head()

,incident_date,policy_state,policy_csl,umbrella_limit,insured_sex,insured_education_level,insured_occupation,insured_hobbies,insured_relationship,capital-gains,...,auto_make,auto_model,auto_year,claim_day_of_policy,location_check,age_group,months_as_customer_groups,policy_annual_premium_groups,policy_deductable_group,fraud_reported
0,2015-01-25,OH,250/500,0.0,MALE,MD,craft-repair,other,husband,53300,...,Saab,92x,2004,100,False,46-50,301-350,medium,501-1000,1
1,2015-01-21,IN,250/500,other,MALE,MD,machine-op-inspct,other,other-relative,0,...,Mercedes,E400,2007,3130,False,41-45,201-250,medium,1501-2000,1
2,2015-02-22,OH,MISSING,other,FEMALE,PhD,sales,other,own-child,35100,...,Dodge,RAM,2007,5282,False,26-30,101-150,medium,1501-2000,0
3,2015-01-10,IL,MISSING,other,FEMALE,PhD,armed-forces,other,unmarried,48900,...,Chevrolet,Tahoe,2014,8996,False,41-45,251-300,medium,1501-2000,1
4,2015-02-17,IL,MISSING,other,MALE,Associate,sales,other,unmarried,66000,...,Accura,RSX,2009,256,False,41-45,201-250,high,501-1000,0


In [9]:
df1_val = df.sort_values(by='incident_date',ascending=False).head(200)
df1_model = df.sort_values(by='incident_date',ascending=False).tail(800)

In [10]:
df1_val = df1_val.drop(["incident_date"],axis=1)
df1_model = df1_model.drop(["incident_date"],axis=1)

In [11]:
df = df.drop(["incident_date"],axis=1)

In [12]:
cat_cols = ['age_group', 'months_as_customer_groups', 'policy_annual_premium_groups','location_check','policy_deductable_group']
for col in cat_cols:
  df1_model[col] = df1_model[col].astype('object')

columns_to_encode = []
for col in df1_model.columns:
  if df1_model[col].dtype == 'object':
    columns_to_encode.append(col)

columns_to_encode

['policy_state',
 'policy_csl',
 'umbrella_limit',
 'insured_sex',
 'insured_education_level',
 'insured_occupation',
 'insured_hobbies',
 'insured_relationship',
 'incident_type',
 'collision_type',
 'incident_severity',
 'authorities_contacted',
 'incident_state',
 'incident_city',
 'property_damage',
 'police_report_available',
 'auto_make',
 'auto_model',
 'location_check',
 'age_group',
 'months_as_customer_groups',
 'policy_annual_premium_groups',
 'policy_deductable_group']

In [13]:
for col in cat_cols:
  df1_val[col] = df1_val[col].astype('object')

Modele ve validationa düşen "auto_model", "location_check", "age_group", "policy_annual_premium_groups" kategorik kolonlarında farklı sayıda unique değerler olduğu için direkt **get_dummies** yaptığımızda farklı sayıda kolonlar oluşuyor. Aşağıda bunu nasıl çözdüğümü anlatıcağım.

Bu problemi **OneHotEncoder**'daki **handle_unknown='ignore'** parametresi yardımıyla çözebildim. OneHotEncoder'ı bu parametreyle modele göre fit edince, modelin kolonlarını alıp sabit tutuyor. Modelde fit edilmiş OneHotEncoder'ı daha sonra validationa gösterdiğimizde ise normalde fazlalık yaratması gereken kolonları (yani validationda olmayan değişkenlerin kolonlarını) yaratmayıp o kolonun encode edilmiş modelde ortaya çıkan kolonlarını otomatik olarak 0'la dolduruyor. İlgili kolonları ayırıp OneHotEncoder uyguladıktan sonra tekrar diğer kolonlarla birleştirmeyle uğraşmamak adına **make_column_transformer** fonksiyonundan yararlandım, bu sayede direkt veriseti üstünden işlem yapabildim. Oluşan yeni verisetini **get_feature_names()** yardımıyla tekrar isimlendirdim. Bu yöntemin modelin görmediği veriye one hot encoding uygulamak için iyi bir yöntem olduğunu düşünüyorum.

In [14]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer

# OneHotEncoder'i fit edip column transformer üzerinden modele uyguladım
ohe = OneHotEncoder(handle_unknown='ignore', dtype="int64")
ct = make_column_transformer((ohe, columns_to_encode), remainder="passthrough")
# yeni bir df oluşturarak transformu gerçekleştirdim ve kolon isimlerini get_feature_names yardımıyla verdim
df2_model = pd.DataFrame(ct.fit_transform(df1_model).toarray(), columns=ct.get_feature_names())

In [15]:
df2_model.head()

,onehotencoder__x0_IL,onehotencoder__x0_IN,onehotencoder__x0_OH,onehotencoder__x1_100/300,onehotencoder__x1_250/500,onehotencoder__x1_500/1000,onehotencoder__x1_MISSING,onehotencoder__x2_0.0,onehotencoder__x2_9999,onehotencoder__x2_other,...,capital-gains,capital-loss,incident_hour_of_the_day,number_of_vehicles_involved,bodily_injuries,witnesses,total_claim_amount,auto_year,claim_day_of_policy,fraud_reported
0,0,0,1,0,0,0,1,1,0,0,...,0,-46200,9,1,2,3,6560,2003,4427,0
1,0,1,0,0,0,0,1,1,0,0,...,0,0,6,1,1,3,6000,2000,8921,0
2,0,1,0,0,0,0,1,0,0,1,...,47200,-69700,12,3,2,3,36300,2013,205,0
3,1,0,0,0,0,0,1,1,0,0,...,26500,0,23,3,0,3,55200,1998,1980,1
4,0,1,0,0,0,0,1,1,0,0,...,0,-31700,0,1,2,1,48290,1995,7692,0


Görüldüğü gibi model de **174** kolon oluşuyor.

In [16]:
# burada önceden model üstünde fit edilmiş column transformeri direkt transformer metoduyla çalıştırdım
df2_val = pd.DataFrame(ct.transform(df1_val).toarray(), columns=ct.get_feature_names())

In [17]:
df2_val.head()

,onehotencoder__x0_IL,onehotencoder__x0_IN,onehotencoder__x0_OH,onehotencoder__x1_100/300,onehotencoder__x1_250/500,onehotencoder__x1_500/1000,onehotencoder__x1_MISSING,onehotencoder__x2_0.0,onehotencoder__x2_9999,onehotencoder__x2_other,...,capital-gains,capital-loss,incident_hour_of_the_day,number_of_vehicles_involved,bodily_injuries,witnesses,total_claim_amount,auto_year,claim_day_of_policy,fraud_reported
0,1,0,0,0,0,0,1,1,0,0,...,57100,0,2,4,0,2,38640,1997,5718,0
1,1,0,0,0,0,0,1,1,0,0,...,0,0,18,3,1,0,42930,2005,3972,0
2,0,0,1,0,0,0,1,1,0,0,...,0,-67000,2,4,0,2,72800,1998,5343,0
3,0,0,1,0,0,0,1,1,0,0,...,0,0,18,3,0,3,34160,2005,3374,1
4,0,0,1,0,0,0,1,1,0,0,...,0,-48800,4,1,2,2,51600,2011,9106,0


Validation setinde de modelle **aynı sayıda**, yani **174** tane kolon oluştu, artık modelimiz validation setiyle test edilebilir.

In [34]:
pd.set_option("display.max_columns", None)
df2_val.head()

,onehotencoder__x0_IL,onehotencoder__x0_IN,onehotencoder__x0_OH,onehotencoder__x1_100/300,onehotencoder__x1_250/500,onehotencoder__x1_500/1000,onehotencoder__x1_MISSING,onehotencoder__x2_0.0,onehotencoder__x2_9999,onehotencoder__x2_other,onehotencoder__x3_FEMALE,onehotencoder__x3_MALE,onehotencoder__x4_Associate,onehotencoder__x4_College,onehotencoder__x4_High School,onehotencoder__x4_JD,onehotencoder__x4_MD,onehotencoder__x4_Masters,onehotencoder__x4_PhD,onehotencoder__x5_adm-clerical,onehotencoder__x5_armed-forces,onehotencoder__x5_craft-repair,onehotencoder__x5_exec-managerial,onehotencoder__x5_farming-fishing,onehotencoder__x5_handlers-cleaners,onehotencoder__x5_machine-op-inspct,onehotencoder__x5_other-service,onehotencoder__x5_priv-house-serv,onehotencoder__x5_prof-specialty,onehotencoder__x5_protective-serv,onehotencoder__x5_sales,onehotencoder__x5_tech-support,onehotencoder__x5_transport-moving,onehotencoder__x6_chess,onehotencoder__x6_cross-fit,onehotencoder__x6_other,onehotencoder__x7_husband,onehotencoder__x7_not-in-family,onehotencoder__x7_other-relative,onehotencoder__x7_own-child,onehotencoder__x7_unmarried,onehotencoder__x7_wife,onehotencoder__x8_Multi-vehicle Collision,onehotencoder__x8_Parked Car,onehotencoder__x8_Single Vehicle Collision,onehotencoder__x8_Vehicle Theft,onehotencoder__x9_?,onehotencoder__x9_Front Collision,onehotencoder__x9_Rear Collision,onehotencoder__x9_Side Collision,onehotencoder__x10_Major Damage,onehotencoder__x10_Minor Damage,onehotencoder__x10_Total Loss,onehotencoder__x10_Trivial Damage,onehotencoder__x11_Ambulance,onehotencoder__x11_Fire,onehotencoder__x11_None,onehotencoder__x11_Other,onehotencoder__x11_Police,onehotencoder__x12_NC,onehotencoder__x12_NY,onehotencoder__x12_OH,onehotencoder__x12_PA,onehotencoder__x12_SC,onehotencoder__x12_VA,onehotencoder__x12_WV,onehotencoder__x13_Arlington,onehotencoder__x13_Columbus,onehotencoder__x13_Hillsdale,onehotencoder__x13_Northbend,onehotencoder__x13_Northbrook,onehotencoder__x13_Riverwood,onehotencoder__x13_Springfield,onehotencoder__x14_?,onehotencoder__x14_NO,onehotencoder__x14_YES,onehotencoder__x15_?,onehotencoder__x15_MISSING,onehotencoder__x15_NO,onehotencoder__x15_YES,onehotencoder__x16_Accura,onehotencoder__x16_Audi,onehotencoder__x16_BMW,onehotencoder__x16_Chevrolet,onehotencoder__x16_Dodge,onehotencoder__x16_Ford,onehotencoder__x16_Honda,onehotencoder__x16_Jeep,onehotencoder__x16_Mercedes,onehotencoder__x16_Nissan,onehotencoder__x16_Saab,onehotencoder__x16_Suburu,onehotencoder__x16_Toyota,onehotencoder__x16_Volkswagen,onehotencoder__x17_3 Series,onehotencoder__x17_92x,onehotencoder__x17_93,onehotencoder__x17_95,onehotencoder__x17_A3,onehotencoder__x17_A5,onehotencoder__x17_Accord,onehotencoder__x17_C300,onehotencoder__x17_CRV,onehotencoder__x17_Camry,onehotencoder__x17_Civic,onehotencoder__x17_Corolla,onehotencoder__x17_E400,onehotencoder__x17_Escape,onehotencoder__x17_F150,onehotencoder__x17_Forrestor,onehotencoder__x17_Fusion,onehotencoder__x17_Grand Cherokee,onehotencoder__x17_Highlander,onehotencoder__x17_Impreza,onehotencoder__x17_Jetta,onehotencoder__x17_Legacy,onehotencoder__x17_M5,onehotencoder__x17_MDX,onehotencoder__x17_ML350,onehotencoder__x17_Malibu,onehotencoder__x17_Maxima,onehotencoder__x17_Neon,onehotencoder__x17_Passat,onehotencoder__x17_Pathfinder,onehotencoder__x17_RAM,onehotencoder__x17_RSX,onehotencoder__x17_Silverado,onehotencoder__x17_TL,onehotencoder__x17_Tahoe,onehotencoder__x17_Ultima,onehotencoder__x17_Wrangler,onehotencoder__x17_X5,onehotencoder__x17_X6,onehotencoder__x18_False,onehotencoder__x18_True,onehotencoder__x19_15-20,onehotencoder__x19_21-25,onehotencoder__x19_26-30,onehotencoder__x19_31-35,onehotencoder__x19_36-40,onehotencoder__x19_41-45,onehotencoder__x19_46-50,onehotencoder__x19_51-55,onehotencoder__x19_56-60,onehotencoder__x19_61-65,onehotencoder__x19_9999,onehotencoder__x20_0-50,onehotencoder__x20_101-150,onehotencoder__x20_151-200,onehotencoder__x20_201-250,onehotencoder__x20_251-300,o

In [35]:
pd.reset_option("display.max_columns")

In [36]:
for i in df2_val.columns:
    

pandas.core.indexes.base.Index

## soru 2

**Soru:** Labelencoding'de ilgili kolon için ölçeklendirme nasıl yapılır? (Verinin doğru etkisiyle sayısal olarak dönüştürülmesi)

**LabelEncoder** featuredan ziyade kategorik target kolonunu encodelamak için önerilir. Eğer feature kısmında kullanılacaksa 2 nominal kategorik değişkenden oluşan(kadın-erkek, doğru-yanlış vb.) kolonlar için kullanılması daha doğru olur, ordinal kategorik değişken kolonları için (çok kötü, kötü, orta, iyi, çok iyi) **OrdinalEncoder** kullanmamız çok daha doğru olur. Eğer kolondaki değişkenler ordinal değilse veya veri seti üstünde etkilerinin mantıklı bir sıralaması yapılamıyorsa kullanmak modelimizi yanıltabilir, o durumda **one hot encoding** kullanmak daha doğru olur, böyle kolonlarda LabelEncoding kullanırsak alfabetik sıraya göre encodelama işlemi yapıyor.

Cinsiyetlerden oluşan bir kolonla basit LabelEncoding uygulaması.

In [18]:
# örnek data oluşturulması
data = {"gender": ["male", "female", "female", "male"]}
df1 = pd.DataFrame(data)
df1

,gender
0,male
1,female
2,female
3,male


In [19]:
# kolona LabelEncoding uygulaması
df1["gender"] = LabelEncoder().fit_transform(df1["gender"])
df1

,gender
0,1
1,0
2,0
3,1


Ordinal bir kolonun sıralamasını tanıtarak **OrdinalEncoder** yardımıyla yaptığım bir encode. Bunun yerine başka yaklaşımlar da uygulamak mümkün, sözlük yardımıyla unique değişkenlerin bir skalasını oluşturup ilgili kolonda replace metoduyla encode yapmak oldukça pratik olabilir.

In [20]:
# örnek data oluşturulması
data2 = {"ordinals": ["medium","very low","high","low","very high","high", "low"]}
df2 = pd.DataFrame(data2)
df2

,ordinals
0,medium
1,very low
2,high
3,low
4,very high
5,high
6,low


In [21]:
df2["ordinals"].unique()

array(['medium', 'very low', 'high', 'low', 'very high'], dtype=object)

In [22]:
# ordinals kolonun unique değerlerinin sıralanmasını bu listeden alacağız
o_categories = ["very low","low","medium","high","very high"]

In [23]:
# yukarda yaptığımız sıralamayı OrdinalEncoder'a tanıtabiliyoruz 
# o yüzden onu kullanıp istediğim sıralamada encoding yapabildim
from sklearn.preprocessing import OrdinalEncoder

enc = OrdinalEncoder(categories=[o_categories], dtype = "int64")
df2["encoded"] = enc.fit_transform(df2[["ordinals"]])
df2[["ordinals","encoded"]]

,ordinals,encoded
0,medium,2
1,very low,0
2,high,3
3,low,1
4,very high,4
5,high,3
6,low,1


## soru 3

**Soru:** Imbalance datasette train test split yaparken neleri göz önünde bulundurmalıyız?

Önemli bir kolonda (genelde hedef kolon dikkate alınır) değişkenler birbirine yakın oranda dağılmadıysa bu veri setine **imbalance dataset** denir. Örneğin bizim derste kullandığımız fraud veri setinde fraud verilerinin oranı no fraud'a göre çok daha az bu yüzden imbalance bir dataset olduğundan bahsettik (yine dersteki gibi fraudları 1'e, no fraudları 0'a çevirdiğimizi farz edelim).

Böyle bir veri setinde train test split yaptığımız zaman traindeki ve testteki 0 ve 1 oranlarına muhakkak dikkat edilmeli. Eğer trainde 1 oranı az olursa modelimiz çoğu duruma 0 demeye çok daha yatkın olur, fazla olursa ise tam tersi. Metodun bölme oranlarını beğenmezsek 1'leri veya 0'ları arttırmak/azaltmak için **oversampling** (SMOTE metoduyla yapılabilir) veya **undersampling** (RandomUnderSampler metoduyla yapılabilir) yapabiliriz.

## soru 4

**Soru:** Validation dataseti (modelin görmediği) nasıl oluşturulur ve nasıl predict etmeye hazır hale getirilir?

Validation dataseti, orijnal datasetimiz encoding, scaling gibi işlemlere girmeden önce belli bir oranda ayrılır. İlgili verisetindeki satırların eğer tarihsel bir sıralaması varsa en yeni tarihli satırları validationa aktarmak daha mantıklı bir davranış olabilir. Validation dataseti modelden bağımsız olarak encodinge ve scalinge sokulmalıdır, ancak modelde nasıl uyguladıysak bu işlemleri validationda da aynı şekilde uygulamalıyız (örneğin birine StandardScaler birine MinMaxScaler uygulamak yanlış olur). Burada dikkat edilecek bir diğer önemli nokta encoding işlemlerinden sonra model ve validation arasında ortaya çıkabilecek farklılıklardır. OneHotEncoding sonrası kolon sayıları değişebilir (1. soruda çözümün anlattım), LabelEncoding sonrası aynı değişkenler farklı değerler alabilir. Encode işlemleri bu tarz sorunlar ortaya çıkmayacak şekilde yapılmalıdır. Bu işlemleri yaptıktan sonra fit edilmiş modele validationdan gelen X ve y (genelde X_val ve y_val olarak isimlendirilir) parametreleri girilerek direkt tahminler ve başarı oranları alınabilir. 6. soruda validationı ayırmayı ve tahmine hazır hale getirmeyi bu durumlara dikkat ederek yaptım, örnek olarak o incelenebilir.

## soru 5

**Soru:** predict_proba metoduyla oran nasıl hesaplanır ve threshold nasıl değiştirilir?

Bu soru için 1. soruda kullandığım verisetini kullanarak derste yapılan örnek modellemelerden birini hızlıca kopyalacağım. Sıradaki 7 kod bloğu modelleme kurulma aşaması, ardından **predict_proba** kısmına gelince orada tekrar açıklamalar yazacağım.

In [24]:
df.head()

,policy_state,policy_csl,umbrella_limit,insured_sex,insured_education_level,insured_occupation,insured_hobbies,insured_relationship,capital-gains,capital-loss,...,auto_make,auto_model,auto_year,claim_day_of_policy,location_check,age_group,months_as_customer_groups,policy_annual_premium_groups,policy_deductable_group,fraud_reported
0,OH,250/500,0.0,MALE,MD,craft-repair,other,husband,53300,0,...,Saab,92x,2004,100,False,46-50,301-350,medium,501-1000,1
1,IN,250/500,other,MALE,MD,machine-op-inspct,other,other-relative,0,0,...,Mercedes,E400,2007,3130,False,41-45,201-250,medium,1501-2000,1
2,OH,MISSING,other,FEMALE,PhD,sales,other,own-child,35100,0,...,Dodge,RAM,2007,5282,False,26-30,101-150,medium,1501-2000,0
3,IL,MISSING,other,FEMALE,PhD,armed-forces,other,unmarried,48900,-62400,...,Chevrolet,Tahoe,2014,8996,False,41-45,251-300,medium,1501-2000,1
4,IL,MISSING,other,MALE,Associate,sales,other,unmarried,66000,-46000,...,Accura,RSX,2009,256,False,41-45,201-250,high,501-1000,0


In [25]:
cat_cols = ['age_group', 'months_as_customer_groups', 'policy_annual_premium_groups','location_check','policy_deductable_group']
for col in cat_cols:
  df[col] = df[col].astype('object')

columns_to_encode = []
for col in df.columns:
  if df[col].dtype == 'object':
    columns_to_encode.append(col)

In [26]:
df3 = pd.get_dummies(df, columns = columns_to_encode)

df3.head()

,capital-gains,capital-loss,incident_hour_of_the_day,number_of_vehicles_involved,bodily_injuries,witnesses,total_claim_amount,auto_year,claim_day_of_policy,fraud_reported,...,months_as_customer_groups_451-500,months_as_customer_groups_51-100,policy_annual_premium_groups_high,policy_annual_premium_groups_low,policy_annual_premium_groups_medium,policy_annual_premium_groups_very high,policy_annual_premium_groups_very low,policy_deductable_group_0-500,policy_deductable_group_1501-2000,policy_deductable_group_501-1000
0,53300,0,5,1,1,2,71610,2004,100,1,...,0,0,0,0,1,0,0,0,0,1
1,0,0,8,1,0,0,5070,2007,3130,1,...,0,0,0,0,1,0,0,0,1,0
2,35100,0,7,3,2,3,34650,2007,5282,0,...,0,0,0,0,1,0,0,0,1,0
3,48900,-62400,5,1,1,2,63400,2014,8996,1,...,0,0,0,0,1,0,0,0,1,0
4,66000,-46000,20,1,0,1,6500,2009,256,0,...,0,0,1,0,0,0,0,0,0,1


In [27]:
X = df3.loc[:, df3.columns != 'fraud_reported']
y = df3['fraud_reported']

In [28]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X = sc.fit_transform(X)

In [29]:
oversample = SMOTE(random_state=9)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3,  random_state = 42)
X_over, y_over = oversample.fit_resample(X_train, y_train)
X_train, X_test, y_train, y_test = train_test_split(X_over, y_over, test_size = 0.3, random_state = 1)

In [30]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
preds = lr.predict(X_test)
score = lr.score(X_test, y_test)
print(score)
print(classification_report(y_test, preds))

0.86875
              precision    recall  f1-score   support

           0       0.89      0.84      0.87       162
           1       0.85      0.90      0.87       158

    accuracy                           0.87       320
   macro avg       0.87      0.87      0.87       320
weighted avg       0.87      0.87      0.87       320



Modellememiz bitti, şimdi sırasıyla 5 **predict** ve 5 **predict_proba** yazdıracağım.

In [31]:
lr.predict(X_test[:5])

array([0, 1, 1, 1, 1])

In [32]:
# predict_proba metoduyla oran hesaplaması bu şekilde yapılabiliyor
lr.predict_proba(X_test[:5])

array([[0.53853819, 0.46146181],
       [0.45657271, 0.54342729],
       [0.09395734, 0.90604266],
       [0.19058162, 0.80941838],
       [0.17427836, 0.82572164]])

Yukarda predict'te gördüklerimiz modelimizin test satırları üstündeki 1 ve 0 tahminleri. Burada ilk tahmininin 0, diğerlerinin 1 olduğu gözüküyor. Altında çalıştırdığımız **predict_proba** kısmında ise bu tahminleri yaparken kullandığı oranları görebiliyoruz. İlk satırın 0 olma olasılığını %53.8, 1 olma olasılığını %46.1 olarak hesaplamış ve 0'ın ihtimali 1'den fazla olduğu için (yani %50'den fazla olduğu için) tahminini 0 olarak yapmış. Oranlar birbirine çok yakın olduğundan pek güvenilir bir tahmin gibi durmuyor bu, 3. satırda ise %90.6'lık 1 tahmini çok yüksek bir oran olduğundan güvenilir olarak adlandırılabilir. Ancak %50'den büyük veya küçük olunca 0-1 yapmasını değil de başka bir orandan büyük veya küçük olunca 0-1 yapmasını istersek (yani thresholdu değiştirmek istersek) yine **predict_proba** yardımıyla bunu yapabiliriz.

In [33]:
# bu şekilde olasılık thresholdunu değiştirebiliyoruz
y_pred = (lr.predict_proba(X_test)[:,1] >= 0.3).astype(bool)
score = lr.score(X_test, y_test)
print(score)
print(classification_report(y_test, y_pred))

0.86875
              precision    recall  f1-score   support

           0       0.99      0.77      0.87       162
           1       0.81      0.99      0.89       158

    accuracy                           0.88       320
   macro avg       0.90      0.88      0.88       320
weighted avg       0.90      0.88      0.88       320



**Threshold**u %30 yapıp ondan büyük olasılıklara 1 demeye başlayınca recall kısmından gördüğümüz gibi neredeyse bütün 1'ler tahmin edilebilmeye başlandı, ancak tahmin edileceği üzere 0'lardaki hatalarımız arttı.